In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

In [2]:
import csv

texts = []
labels_ = []

# Open the CSV file and read it line by line
with open("/content/complaints.csv", "r", encoding="utf-8") as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    # Skip the header line
    next(csv_reader)
    # Iterate over each row in the CSV file
    for row in csv_reader:
      # Extract text and label from the row
      text = row[0]  # Remove leading and trailing quotes
      if len(row) > 1:
        label = row[1]  # Extract label from the second column
      else:
        label = None
      # Append text and label to the respective lists
      texts.append(text)
      labels_.append(label)

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:

# Preprocess the text data
texts = [word_tokenize(text.lower()) for text in texts]

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_)


In [5]:
# Convert texts to sequences and pad them
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [6]:
# Define model parameters
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
num_classes = len(label_encoder.classes_)

# Define the CNN model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
3535/3535 [==============================] - 196s 54ms/step - loss: 0.3494 - accuracy: 0.8714 - val_loss: 0.2946 - val_accuracy: 0.8906
Epoch 2/10
3535/3535 [==============================] - 82s 23ms/step - loss: 0.2434 - accuracy: 0.9102 - val_loss: 0.3010 - val_accuracy: 0.8863
Epoch 3/10
3535/3535 [==============================] - 71s 20ms/step - loss: 0.1790 - accuracy: 0.9352 - val_loss: 0.2993 - val_accuracy: 0.8934
Epoch 4/10
3535/3535 [==============================] - 68s 19ms/step - loss: 0.1237 - accuracy: 0.9563 - val_loss: 0.3534 - val_accuracy: 0.8840
Epoch 5/10
3535/3535 [==============================] - 65s 18ms/step - loss: 0.0857 - accuracy: 0.9706 - val_loss: 0.3906 - val_accuracy: 0.8921
Epoch 6/10
3535/3535 [==============================] - 64s 18ms/step - loss: 0.0637 - accuracy: 0.9784 - val_loss: 0.4471 - val_accuracy: 0.8895
Epoch 7/10
3535/3535 [==============================] - 64s 18ms/step - loss: 0.0506 - accuracy: 0.9829 - val_loss: 0.5206 

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

2209/2209 [==============================] - 8s 3ms/step - loss: 0.6092 - accuracy: 0.8914
Test Loss: 0.6091961860656738, Test Accuracy: 0.891366183757782


In [8]:
# Predict the probabilities for the test data
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert encoded labels back to original labels
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)

# Generate classification report
report = classification_report(y_test_original, y_pred_original)
print(report)

2209/2209 [==============================] - 6s 3ms/step
                                      precision    recall  f1-score   support

   Attempts to collect debt not owed       0.77      0.76      0.77     14477
               Communication tactics       0.83      0.72      0.77      4160
                       Fraud or scam       0.93      0.83      0.88      2391
Incorrect information on your report       0.93      0.95      0.94     46122
          Struggling to pay mortgage       0.89      0.94      0.91      3537

                            accuracy                           0.89     70687
                           macro avg       0.87      0.84      0.85     70687
                        weighted avg       0.89      0.89      0.89     70687

